In [0]:
import random
import json
import traceback

import numpy as np
import pandas as pd
import great_expectations as ge

In [0]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

In [0]:
import glob
G = glob.glob('/Users/abe/Documents/superconductive/tools/maglev/autoinspection/data/vincent_is_awesome/*.csv')

for g in G:
    df = ge.read_csv(g)
    for col in df.columns:
        result = df.expect_column_proportion_of_unique_values_to_be_between(col, 0)
        if result["true_value"] < .05:
            print g, col

In [0]:
len(G)

In [0]:
# df = ge.read_csv('/Users/abe/Documents/superconductive/tools/maglev/autoinspection/data/vincent_is_awesome/car_Florida.csv')
# df = ge.read_csv('/Users/abe/Documents/superconductive/tools/maglev/autoinspection/data/vincent_is_awesome/boot_amis.csv')
# g = '/Users/abe/Documents/superconductive/tools/maglev/autoinspection/data/vincent_is_awesome/boot_nuclear.csv'
# g = '/Users/abe/Documents/superconductive/tools/maglev/autoinspection/data/vincent_is_awesome/boot_acme.csv'
# g = '/Users/abe/Documents/superconductive/tools/maglev/autoinspection/data/vincent_is_awesome/COUNT_rwm5yr.csv'
g = '/Users/abe/Documents/superconductive/tools/maglev/autoinspection/data/vincent_is_awesome/COUNT_rwm5yr.csv'
# g = random.choice(G)
df = ge.read_csv(g)
print g
print df.shape

In [0]:
df.head()

In [0]:
var1 = 'hospvis'
var2 = 'hhninc'

# print pd.crosstab(df[var1], df[var2])
# x = df[var1].value_counts()
# print x/x.sum()
# x = df[var2].value_counts()
# print x/x.sum()

df[var1].hist(bins=20, alpha=.7)
df[var2].hist(bins=20, alpha=.7)
# plt.scatter(df[var1], df[var2])

In [0]:
rows = []
for k,g in enumerate(G):
    print k,g
    try:
        df = ge.read_csv(g)
    except:
        continue
    
    if df.shape[0]<40 or df.shape[1]>40:
        continue

    for i, i_col in list(enumerate(df.columns))[1:]:
        if df[i_col].value_counts().shape < 30:
            continue

        for j, j_col in list(enumerate(df.columns))[i:]:
    #         print i_col, j_col
            if df[j_col].value_counts().shape < 30:
                continue
                
            try:
                df["random"] = df[i_col].map(lambda x: random.random())
                df_training = df[df.random > .5]
                df_testing = ge.dataset.PandasDataSet(df[df.random <= .5])

                partition = ge.dataset.util.continuous_partition_data(df_training[i_col], bins='ntile')
                new_partition = ge.dataset.util.continuous_partition_data(df_training[j_col], bins='ntile')

                same_result = df_testing.expect_column_kl_divergence_less_than(
                    i_col,
                    partition,
                    20,
    #                 tail_weight_holdout=0.0001,
#                     tail_weight_holdout=0.01,
#                     tail_weight_holdout=0.1,
                    tail_weight_holdout=1.0/df_testing.shape[0]
                )
                diff_result = df_testing.expect_column_kl_divergence_less_than(
                    j_col,
                    partition,
                    20,
#                     tail_weight_holdout=0.0001,
#                     tail_weight_holdout=0.01,
#                     tail_weight_holdout=0.1,
                    tail_weight_holdout=1.0/df_testing.shape[0]
                )
                if i == j:
                    diff_value = None
                else:
                    diff_value = diff_result['true_value']

                row = {
                    'column_A' : i_col,
                    'column_B' : j_col,
                    'same_value' : same_result['true_value'],
                    'diff_value' : diff_value,
                    'source_file' : g,
                }
        #         same_value = result['true_value']
        #         values.append(value)
        #         print col, value
                rows.append(row)

            except Exception, err:
                pass
#                 print i_col, j_col
#                 traceback.print_exc()


result_df = pd.DataFrame(rows)
# plt.hist(values, bins=20)

In [0]:
result_df = pd.DataFrame(rows)
result_df.shape

In [0]:
result_df.diff_value[result_df.diff_value<np.inf].hist(alpha=.5, color='red', bins=20)
result_df.same_value[result_df.same_value<np.inf].hist(alpha=.5, bins=20)

In [0]:
np.log(result_df.same_value)[result_df.same_value<np.inf].hist(alpha=.5, bins=20)

In [0]:
rows = []
for i in range(0,200):
    k = i*1./100

    row = {
        "pct_false_negative" : (result_df.same_value>k).mean(),
        "pct_false_positive" : (result_df.diff_value<k).mean(),
        "k" : k
    }
    rows.append(row)

false_pos_df = pd.DataFrame(rows)

In [0]:
#0.44 .05
#0.75 .01
false_pos_df[false_pos_df.pct_false_negative<.01]

In [0]:
plt.scatter(false_pos_df.pct_false_negative, false_pos_df.pct_false_positive)
plt.xlabel("% False Positive")
plt.ylabel("% False Negative")

In [0]:
import sklearn
from sklearn.neighbors import KernelDensity

In [0]:
result_df.diff_value[result_df.diff_value<np.inf].plot('kde')
result_df.same_value[result_df.same_value<np.inf].plot('kde')

In [0]:
diff_kde = KernelDensity()
#.hist(alpha=.5, color='red', bins=20)
diff_kde.fit(result_df.diff_value[result_df.diff_value<np.inf])

same_kde = KernelDensity()
#.hist(alpha=.5, bins=20)
same_kde.fit(result_df.same_value[result_df.same_value<np.inf])

In [0]:
result_df.sort_values('diff_value').head(20)

In [0]:
result_df.ix[1979].to_dict()

In [0]:
df.head()

In [0]:
df.shape

In [0]:
plt.scatter(df.GORE, df.BUSH)
plt.plot([0,400000], [0,400000])

In [0]:
pd.plotting.scatter_matrix(df)

In [0]:
df.shape

In [0]:
df["lnBUSH"] = np.log(df.BUSH)

In [0]:
# ge.dataset.util.categorical_partition_data
my_part = ge.dataset.util.continuous_partition_data(df.lnBUSH)
# my_part = ge.dataset.util.categorical_partition_data(df.BUSH)
# my_part = ge.dataset.util.kde_partition_data(df.BUSH)

#ge.generate_continuous_partition_from_series(df.BUSH)
#df.BUSH.generate_continuous_partition_from_series()

In [0]:
df.expect_column_kl_divergence_less_than('lnBUSH', my_part, 1)

In [0]:
# plt.plot(my_part["partition"], my_part["weights"])
print json.dumps(my_part, indent=2)

In [0]:
plt.bar(my_part["partition"][:-1], my_part["weights"])

In [0]:
np.log(df.BUSH).hist()

In [0]:
df = ge.read_csv('/Users/abe/Documents/superconductive/tools/maglev/autoinspection/data/vincent_is_awesome/boot_channing.csv')

In [0]:
df.head()

In [0]:
?ge.dataset.util.continuous_partition_data(df.age)

In [0]:
ge.dataset.util.categorical_partition_data(df.gender)

In [0]:
import random

In [0]:
?df.expect_column_kl_divergence_less_than

In [0]:
df.age.hist()

In [0]:
col = 'age'
values = []
for i in range(100):
#     print "#"*80
    df["random"] = df.gender.map(lambda x: random.random())
    df_training = df[df.random > .5]
    df_testing = ge.dataset.PandasDataSet(df[df.random <= .5])
    
#     partition = ge.dataset.util.continuous_partition_data(df_training[col], bins='ntile')
    partition = ge.dataset.util.continuous_partition_data(df_training[col], bins='ntile')
    new_partition = ge.dataset.util.continuous_partition_data(df_testing[col], bins='ntile')
#     print partition

#     plt.scatter(partition['partition'], np.hstack([partition['weights'], partition['weights'][-1:]]))
#     plt.plot(partition['partition'], np.hstack([partition['weights'], partition['weights'][-1:]]))
#     plt.plot(new_partition['partition'], np.hstack([new_partition['weights'], new_partition['weights'][-1:]]))
#     plt.show()

    result = df_testing.expect_column_kl_divergence_less_than(
        col,
        partition,
        20,
#         internal_weight_holdout=.5,
        tail_weight_holdout=1.0/df_testing.shape[0]
    )
    value = result['true_value']
    values.append(value)
#     print value

plt.hist(values, bins=20)

In [0]:
df.head()

In [0]:
col = 'age'
values = []
for col in ['age', 'obs', 'death', 'ageentry', 'time', 'gender', 'random']:
    df["random"] = df.gender.map(lambda x: random.random())
    df_training = df[df.random > .5]
    df_testing = ge.dataset.PandasDataSet(df[df.random <= .5])

    partition = ge.dataset.util.continuous_partition_data(df_training['age'], bins='ntile')
    new_partition = ge.dataset.util.continuous_partition_data(df_training[col], bins='ntile')

    result = df_testing.expect_column_kl_divergence_less_than(
        col,
        partition,
        20,
#         internal_weight_holdout=.5,
        tail_weight_holdout=1.0/df_testing.shape[0]
    )
    value = result['true_value']
    values.append(value)
    print col, value

plt.hist(values, bins=20)

In [0]:
rows = []
for k,g in enumerate(G):
    print k,g
    try:
        df = ge.read_csv(g)
    except:
        continue
    
    if df.shape[0]<40 or df.shape[1]>40:
        continue

    for i, i_col in list(enumerate(df.columns))[1:]:
        if df[i_col].value_counts().shape < 30:
            continue

        for j, j_col in list(enumerate(df.columns))[i:]:
    #         print i_col, j_col
            if df[j_col].value_counts().shape < 30:
                continue
                
            try:
                df["random"] = df[i_col].map(lambda x: random.random())
                df_training = df[df.random > .5]
                df_testing = ge.dataset.PandasDataSet(df[df.random <= .5])

                partition = ge.dataset.util.categorical_partition_data(df_training[i_col])
                new_partition = ge.dataset.util.categorical_partition_data(df_training[j_col])

                same_result = df_testing.expect_column_chisquare_test_p_value_greater_than(
                    i_col,
                    partition,
                    tail_weight_holdout=1.0/df_testing.shape[0]
                )
                diff_result = df_testing.expect_column_chisquare_test_p_value_greater_than(
                    j_col,
                    partition,
                    tail_weight_holdout=1.0/df_testing.shape[0]
                )
                if i == j:
                    diff_value = None
                else:
                    diff_value = diff_result['true_value']

#                 print same_result

                row = {
                    'column_A' : i_col,
                    'column_B' : j_col,
                    'same_value' : same_result['true_value'],
                    'diff_value' : diff_value,
                    'source_file' : g,
                }
        #         same_value = result['true_value']
        #         values.append(value)
        #         print col, value
                rows.append(row)

            except Exception, err:
#                 pass
                print i_col, j_col
                traceback.print_exc()


chi_result_df = pd.DataFrame(rows)
# plt.hist(values, bins=20)

In [0]:
chi_result_df = pd.DataFrame(rows)

In [0]:
chi_result_df.shape

In [0]:
chi_result_df.same_value[chi_result_df.same_value<np.inf].hist(bins=20)

In [0]:
chi_result_df.same_value.hist(bins=20)

In [0]:
(chi_result_df.same_value<.05).mean()

In [0]:
partition

In [0]:
?ge.dataset.util.continuous_partition_data(df_training[i_col], bin="ntile")

In [0]:
result_df.same_value[result_df.same_value<np.inf].hist()

In [0]:
df["random"] = df[i_col].map(lambda x: random.random())
df_training = df[df.random > .5]
df_testing = ge.dataset.PandasDataSet(df[df.random <= .5])

partition = ge.dataset.util.categorical_partition_data(df_training[i_col])
new_partition = ge.dataset.util.categorical_partition_data(df_training[j_col])

same_result = df_testing.expect_column_chisquare_test_p_value_greater_than(
    i_col,
    partition,
)
diff_result = df_testing.expect_column_chisquare_test_p_value_greater_than(
    j_col,
    partition,
)
if i == j:
    diff_value = None
else:
    diff_value = diff_result['true_value']

#                 print same_result

row = {
    'column_A' : i_col,
    'column_B' : j_col,
    'same_value' : same_result['true_value'],
    'diff_value' : diff_value,
    'source_file' : g,
}
#         same_value = result['true_value']
#         values.append(value)
#         print col, value
rows.append(row)